In [279]:
# # Small LSTM Network to Generate Text for Alice in Wonderland

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils
# # load ascii text and covert to lowercase
# # filename = "wonderland.txt"
# # raw_text = open(filename).read()
# raw_text = list
# # create mapping of unique chars to integers
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print ("Total Characters: ", n_chars)
# print ("Total Vocab: ", n_vocab)
# # prepare the dataset of input to output pairs encoded as integers
# seq_length = 100
# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
#     seq_in = raw_text[i:i + seq_length]
#     seq_out = raw_text[i + seq_length]
#     dataX.append([char_to_int[char] for char in seq_in])
#     dataY.append(char_to_int[seq_out])
# n_patterns = len(dataX)
# print ("Total Patterns: ", n_patterns)
# # reshape X to be [samples, time steps, features]
# X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# # normalize
# X = X / float(n_vocab)
# # one hot encode the output variable
# y = np_utils.to_categorical(dataY)
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# # define the checkpoint
# filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
# # fit the model
# model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [288]:
import numpy as np
import re, os, pygtrie, numbers
dimension = 29
def readFile():
    list = []
    for i in os.listdir("assignment1-data/"):
        if i.startswith("training"):
            input = open("assignment1-data/"+ i, 'r')
            list.append(input.readlines())
    return(list)
def process_line(string):
    string = re.sub(r'[^A-Za-z0-9\ \.]', "", string)
    string = re.sub('[0-9]', "0",string)
    line.strip('\n')
    string = string.lower()
    return(string)
processed_lines = []
list = readFile()
for l in list:
    for line in l:
        processed_lines.append(process_line(line))
#     print('-------------------')
# print(processed_lines)
# for processed_line in processed_lines:
print(processed_lines[0])

triGram = np.zeros((29,29,29))
# print(processed_lines)
for processed_line in processed_lines:
    constructTriGram(processed_line.strip(' ')) # shift it from the for above
print('done')

wiederaufnahme der sitzungsperiode
wie
ied
ede
der
era
rau
auf
ufn
fna
nah
ahm
hme
me 
e d
 de
der
er 
r s
 si
sit
itz
tzu
zun
ung
ngs
gsp
spe
per
eri
rio
iod
ode
ich
ch 
h e
 er
erk
rkl
klr
lre
re 
e d
 di
die
ie 
e a
 am
am 
m f
 fr
fre
rei
eit
ita
tag
ag 
g d
 de
dem
em 
m 0
 00
00.
0. 
. d
 de
dez
eze
zem
emb
mbe
ber
er 
r u
 un
unt
nte
ter
erb
rbr
bro
roc
och
che
hen
ene
ne 
e s
 si
sit
itz
tzu
zun
ung
ngs
gsp
spe
per
eri
rio
iod
ode
de 
e d
 de
des
es 
s e
 eu
eur
uro
rop
opi
pis
isc
sch
che
hen
en 
n p
 pa
par
arl
rla
lam
ame
men
ent
nts
ts 
s f
 fr
fr 
r w
 wi
wie
ied
ede
der
era
rau
auf
ufg
fge
gen
eno
nom
omm
mme
men
en 
n w
 wn
wns
nsc
sch
che
he 
e i
 ih
ihn
hne
nen
en 
n n
 no
noc
och
chm
hma
mal
als
ls 
s a
 al
all
lle
les
es 
s g
 gu
gut
ute
te 
e z
 zu
zum
um 
m j
 ja
jah
ahr
hre
res
esw
swe
wec
ech
chs
hse
sel
el 
l u
 un
und
nd 
d h
 ho
hof
off
ffe
fe 
e d
 da
da 
a s
 si
sie
ie 
e s
 sc
sch
chn
hne
ne 
e f
 fe
fer
eri
rie
ien
en 
n h
 ha
hat
att
tte
ten
en.
wie
ie 
e

seh
ehr
hr 
r v
 vi
vie
iel
ele
len
en 
n e
 en
ent
nts
tsc
sch
chl
hli
lie
ieu
eun
ung
nge
gen
en 
n f
 fe
fes
est
stg
tge
ges
est
ste
tel
ell
llt
lt 
t  
  g
 ge
ger
era
rad
ade
de 
e w
 wh
whr
hre
ren
end
nd 
d d
 de
der
er 
r l
 le
let
etz
tzt
zte
ten
en 
n p
 pl
ple
len
ena
nar
art
rta
tag
agu
gun
ung
ng 
g d
 de
des
es 
s v
 ve
ver
erg
rga
gan
ang
nge
gen
ene
nen
en 
n j
 ja
jah
ahr
hre
res
es 
s d
 da
da 
a d
 di
die
ies
es 
s n
 ni
nic
ich
cht
ht 
t n
 nu
nur
ur 
r e
 ei
ein
in 
n j
 ju
jur
uri
ris
ist
sti
tis
isc
sch
che
her
er 
r f
 fa
fal
all
ll 
l i
 is
ist
st 
t u
 un
und
nd 
d d
 da
da 
a e
 es
es 
s f
 fa
fal
als
lsc
sch
ch 
h i
 is
ist
st 
t a
 al
ale
lex
exa
xan
and
nde
der
er 
r n
 ni
nik
iki
kit
iti
tin
in 
n k
 kr
kri
rim
imi
min
ina
nal
ali
lit
itt
tt 
t u
 un
und
nd 
d v
 ve
ver
err
rra
rat
at 
t v
 vo
vor
orz
rzu
zuw
uwe
wer
erf
rfe
fen
en 
n w
 we
wei
eil
il 
l w
 wi
wir
ir 
r a
 al
als
ls 
s b
 be
bet
etr
tro
rof
off
ffe
fen
ene
ne 
e v
 vo
von
on 
n s
 se
sei


lez
ez 
z u
 un
und
nd 
d i
 ic
ich
ch 
h h
 ha
hat
att
tte
ten
en 
n e
 ei
ein
ini
nig
ige
ge 
e a
 an
anf
nfr
fra
rag
age
gen
en 
n z
 zu
zu 
u b
 be
bes
est
sti
tim
imm
mmt
mte
ten
en 
n i
 in
in 
n e
 ei
ein
ine
ner
er 
r s
 sp
spa
pan
ani
nis
isc
sch
che
hen
en 
n z
 ze
zei
eit
itu
tun
ung
ng 
g w
 wi
wie
ied
ede
der
erg
rge
geg
ege
geb
ebe
ben
ene
nen
en 
n s
 st
ste
tel
ell
llu
lun
ung
ngn
gna
nah
ahm
hme
men
en 
n d
 de
der
er 
r v
 vi
viz
ize
zep
epr
prs
rsi
sid
ide
den
ent
nti
tin
in 
n f
 fr
fra
rau
au 
u d
 de
de 
e p
 pa
pal
ala
lac
aci
cio
io 
o g
 ge
ges
est
ste
tel
ell
llt
lt.
die
ie 
e z
 zu
zus
ust
stn
tnd
ndi
dig
ige
gen
en 
n d
 di
die
ien
ens
nst
ste
te 
e h
 ha
hab
abe
ben
en 
n s
 si
sie
ie 
e n
 ni
nic
ich
cht
ht 
t i
 in
in 
n d
 di
die
ie 
e t
 ta
tag
age
ges
eso
sor
ord
rdn
dnu
nun
ung
ng 
g a
 au
auf
ufg
fge
gen
eno
nom
omm
mme
men
en 
n d
 da
da 
a s
 si
sie
ie 
e d
 de
der
er 
r m
 me
mei
ein
inu
nun
ung
ng 
g w
 wa
war
are
ren
en 
n s
 si
sie
ie 
e s
 se


nac
ach
ch 
h v
 ve
ver
erf
rfa
fah
ahr
hre
ren
en 
n w
 wi
wir
ir 
r w
 wi
wie
ie 
e b
 bl
bli
lic
ich
ch 
h e
 ei
ein
in 
n r
 re
red
edn
dne
ner
er 
r d
 da
daf
afr
fr 
r e
 ei
ein
ine
ner
er 
r d
 da
dag
age
geg
ege
gen
en.
fra
rau
au 
u p
 pr
prs
rsi
sid
ide
den
ent
nti
tin
die
ie 
e v
 vo
vor
ors
rst
ste
tel
ell
llu
lun
ung
ng 
g d
 de
des
es 
s p
 po
pol
oli
lit
iti
tis
isc
sch
che
hen
en 
n p
 pr
pro
rog
ogr
gra
ram
amm
mms
ms 
s d
 de
der
er 
r k
 ko
kom
omm
mmi
mis
iss
ssi
sio
ion
on 
n p
 pr
pro
rod
odi
di 
i f
 fr
fr 
r d
 di
die
ie 
e g
 ge
ges
esa
sam
amt
mte
te 
e w
 wa
wah
ahl
hlp
lpe
per
eri
rio
iod
ode
de 
e g
 gi
gin
ing
ng 
g a
 au
auf
uf 
f e
 ei
ein
ine
nen
en 
n v
 vo
vor
ors
rsc
sch
chl
hla
lag
ag 
g d
 de
der
er 
r f
 fr
fra
rak
akt
kti
tio
ion
on 
n d
 de
der
er 
r s
 so
soz
ozi
zia
ial
ald
lde
dem
emo
mok
okr
kra
rat
ati
tis
isc
sch
che
hen
en 
n p
 pa
par
art
rte
tei
ei 
i e
 eu
eur
uro
rop
opa
pas
as 
s z
 zu
zur
urc
rck
ck 
k d
 de
der
er 
r d
 di
die
ie 


er 
r k
 ko
kom
omm
mmi
mis
iss
ssi
sio
ion
on 
n i
 in
in 
n s
 sc
sch
chr
hri
rif
ift
ftl
tli
lic
ich
che
her
er 
r f
 fo
for
orm
rm 
m e
 ei
ein
ine
ne 
e w
 wo
woc
och
che
he 
e v
 vo
vor
orh
rhe
her
er 
r  
  u
 un
und
nd 
d n
 ni
nic
ich
cht
ht 
t w
 wi
wie
ie 
e v
 ve
ver
ere
rei
ein
inb
nba
bar
art
rt 
t a
 am
am 
m t
 ta
tag
ag 
g z
 zu
zuv
uvo
vor
or 
r  
  z
 zu
zur
ur 
r k
 ke
ken
enn
nnt
ntn
tni
nis
is 
s z
 zu
zu 
u g
 ge
geb
ebe
ben
en 
n w
 wo
wob
obe
bei
ei 
i z
 zu
zu 
u b
 be
ber
erc
rck
cks
ksi
sic
ich
cht
hti
tig
ige
gen
en 
n i
 is
ist
st 
t d
 da
da 
a d
 da
das
as 
s l
 le
leg
egi
gis
isl
sla
lat
ati
tiv
ivp
vpr
pro
rog
ogr
gra
ram
amm
mm 
m i
 im
im 
m f
 fe
feb
ebr
bru
rua
uar
ar 
r d
 di
dis
isk
sku
kut
uti
tie
ier
ert
rt 
t w
 we
wer
erd
rde
den
en 
n w
 wi
wir
ird
rd 
d s
 so
so 
o d
 da
da 
a w
 wi
wir
ir 
r a
 au
auf
uf 
f d
 di
die
ie 
e a
 au
aus
uss
ssp
spr
pra
rac
ach
che
he 
e v
 ve
ver
erz
rzi
zic
ich
cht
hte
ten
en 
n k
 kn
knn
nnt
nte
ten
en 
n d


...

ste
tem
eme
me 
e d
 de
der
er 
r k
 ko
kom
omm
mmi
mis
iss
ssi
sio
ion
on.
ich
ch 
h k
 ka
kan
ann
nn 
n d
 de
dem
em 
m e
 eu
eur
uro
rop
opi
pis
isc
sch
che
hen
en 
n p
 pa
par
arl
rla
lam
ame
men
ent
nt 
t d
 di
die
ies
esb
sbe
bez
ezg
zgl
gli
lic
ich
ch 
h m
 mi
mit
itt
tte
tei
eil
ile
len
en 
n d
 da
da 
a i
 ic
ich
ch 
h m
 mi
mit
itt
tte
te 
e d
 de
des
es 
s j
 ja
jah
ahr
hre
res
es 
s 0
 00
000
000
00 
0 b
 be
bea
eab
abs
bsi
sic
ich
cht
hti
tig
ige
ge 
e m
 mi
mit
it 
t d
 de
den
en 
n n
 na
nat
ati
tio
ion
ona
nal
ale
len
en 
n u
 un
und
nd 
d r
 re
reg
egi
gio
ion
ona
nal
ale
len
en 
n b
 be
beh
ehr
hrd
rde
den
en 
n e
 ei
ein
in 
n s
 se
sem
emi
min
ina
nar
ar 
r b
 be
ber
er 
r e
 eb
ebe
ben
en 
n d
 di
die
ies
ese
se 
e f
 fr
fra
rag
age
ge 
e d
 de
der
er 
r b
 be
bew
ewe
wer
ert
rtu
tun
ung
ng 
g d
 de
der
er 
r v
 ve
ver
erf
rfa
fah
ahr
hre
ren
en 
n f
 fr
fr 
r d
 de
den
en 
n a
 au
aus
ust
sta
tau
aus
usc
sch
ch 
h v
 vo
von
on 
n b
 be
bew
ewh
whr
hrt
rte
ten
en 


 au
auc
uch
ch 
h u
 um
um 
m m
 mi
mit
it 
t d
 de
den
en 
n b
 br
brg
rge
ger
ern
rn 
n m
 mi
mit
it 
t i
 ih
ihr
hre
ren
en 
n g
 ge
gew
ewh
whl
hlt
lte
ten
en 
n v
 ve
ver
ert
rtr
tre
ret
ete
ter
ern
rn 
n m
 mi
mit
it 
t i
 ih
ihn
hne
nen
en 
n e
 ei
ein
ine
ne 
e f
 ff
ffe
fen
ent
ntl
tli
lic
ich
che
he 
e a
 au
aus
uss
ssp
spr
pra
rac
ach
che
he 
e b
 be
ber
er 
r u
 un
uns
nse
ser
ere
re 
e r
 re
reg
egi
gio
ion
ona
nal
alp
lpo
pol
oli
lit
iti
tik
ik 
k u
 un
und
nd 
d b
 be
ber
er 
r d
 da
das
as 
s w
 wa
was
as 
s e
 ei
ein
ine
nes
es 
s t
 ta
tag
age
ges
es 
s e
 ei
ein
ine
ne 
e e
 eu
eur
uro
rop
opi
pis
isc
sch
che
he 
e r
 ra
rau
aum
umo
mor
ord
rdn
dnu
nun
ung
ngs
gsp
spo
pol
oli
lit
iti
tik
ik 
k w
 we
wer
erd
rde
den
en 
n k
 kn
knn
nnt
nte
te 
e z
 zu
zu 
u f
 fh
fhr
hre
ren
en.
jed
ede
den
enf
nfa
fal
all
lls
ls 
s k
 ko
kon
onn
nnt
nte
te 
e i
 ic
ich
ch 
h f
 fe
fes
est
sts
tst
ste
tel
ell
lle
len
en 
n d
 da
da 
a s
 si
sie
ie 
e d
 de
den
en 
n w
 wu
wun
uns
nsc


uns
ns 
s z
 zu
zur
ur 
r v
 ve
ver
erf
rfg
fgu
gun
ung
ng 
g s
 st
ste
teh
ehe
hen
end
nde
den
en 
n f
 fi
fin
ina
nan
anz
nzi
zin
ins
nst
str
tru
rum
ume
men
ent
nte
ten
en 
n b
 be
ber
ere
rei
eit
its
ts 
s e
 ei
ein
ine
ne 
e p
 po
pol
oli
lit
iti
tik
ik 
k a
 ab
abz
bze
zei
eic
ich
chn
hne
net
et 
t d
 di
die
ie 
e s
 si
sic
ich
ch 
h f
 fr
fr 
r d
 di
die
ie 
e e
 er
ers
rst
ste
ten
en 
n l
 ln
lnd
nde
der
er 
r d
 di
die
ie 
e z
 zu
zu 
u u
 un
uns
nse
ser
ere
rer
er 
r g
 ge
gem
eme
mei
ein
ins
nsc
sch
cha
haf
aft
ft 
t s
 st
sto
toe
oen
en 
n w
 we
wer
erd
rde
den
en 
n z
 zu
zu 
u e
 ei
ein
ine
ner
er 
r k
 ko
koh
ohs
hsi
sio
ion
ons
nsp
spo
pol
oli
lit
iti
tik
ik 
k e
 en
ent
ntw
twi
wic
ick
cke
kel
eln
ln 
n k
 kn
knn
nnt
nte
te.
ich
ch 
h d
 de
den
enk
nke
ke 
e h
 hi
hie
ier
er 
r i
 in
ins
nsb
sbe
bes
eso
son
ond
nde
der
ere
re 
e a
 an
an 
n d
 da
das
as 
s s
 st
str
tru
ruk
ukt
ktu
tur
urp
rpo
pol
oli
lit
iti
tis
isc
sch
che
he 
e i
 in
ins
nst
str
tru
rum
ume
men
ent


ese
se 
e e
 en
ent
nts
tsp
spr
pre
rec
ech
che
hen
end
nd 
d d
 di
dis
isk
sku
kut
uti
tie
ier
ere
ren
en.
ich
ch 
h m
 mc
mch
cht
hte
te 
e d
 da
da 
a w
 wi
wir
ir 
r d
 da
dar
ara
rau
aus
us 
s e
 ei
ein
ine
ne 
e e
 el
ele
lem
eme
men
ent
nta
tar
are
re 
e l
 le
leh
ehr
hre
re 
e z
 zi
zie
ieh
ehe
hen
en.
wen
enn
nn 
n k
 kn
knf
nft
fti
tig
ig 
g w
 wi
wic
ich
cht
hti
tig
ige
ge 
e d
 de
deb
eba
bat
att
tte
ten
en 
n z
 zu
zu 
u b
 be
bes
est
sti
tim
imm
mmt
mte
ten
en 
n t
 th
the
hem
eme
men
en 
n z
 zw
zwi
wis
isc
sch
che
hen
en 
n d
 di
die
ies
ese
sem
em 
m h
 ha
hau
aus
us 
s u
 un
und
nd 
d d
 de
der
er 
r e
 eu
eur
uro
rop
opi
pis
isc
sch
che
hen
en 
n k
 ko
kom
omm
mmi
mis
iss
ssi
sio
ion
on 
n a
 an
ans
nst
ste
teh
ehe
hen
en 
n d
 da
dan
ann
nn 
n s
 so
sol
oll
llt
lte
ten
en 
n w
 wi
wir
ir 
r w
 we
wen
eni
nig
igs
gst
ste
ten
ens
ns 
s e
 ei
ein
ine
nen
en 
n v
 vo
vol
oll
lle
len
en 
n a
 ar
arb
rbe
bei
eit
its
tsm
smo
mon
ona
nat
at 
t v
 vo
vor
orh
rhe
her
er 
r k


 im
im 
m n
 na
nam
ame
men
en 
n d
 de
des
es 
s a
 au
aus
uss
ssc
sch
chu
hus
uss
sse
ses
es 
s f
 fr
fr 
r w
 wi
wir
irt
rts
tsc
sch
cha
haf
aft
ft 
t u
 un
und
nd 
d w
 wh
whr
hru
run
ung
ng 
g z
 zu
zum
um 
m x
 xx
xxv
xvi
vii
iii
ii.
ber
eri
ric
ich
cht
ht 
t d
 de
der
er 
r k
 ko
kom
omm
mmi
mis
iss
ssi
sio
ion
on 
n s
 se
sek
ek0
k00
000
000
00 
0 0
 00
000
00 
0  
  c
 c0
c00
000
000
000
000
000
000
000
00 
0  
  0
 00
000
000
000
000
000
000
00c
0co
cos
os 
s b
 be
ber
er 
r d
 di
die
ie 
e w
 we
wet
ett
ttb
tbe
bew
ewe
wer
erb
rbs
bsp
spo
pol
oli
lit
iti
tik
ik 
k 0
 00
000
000
a00
000
000
000
000
000
000
000
00 
0 v
 vo
von
on 
n h
 he
her
err
rrn
rn 
n j
 jo
jon
onc
nck
ckh
khe
hee
eer
er 
r i
 im
im 
m n
 na
nam
ame
men
en 
n d
 de
des
es 
s a
 au
aus
uss
ssc
sch
chu
hus
uss
sse
ses
es 
s f
 fr
fr 
r w
 wi
wir
irt
rts
tsc
sch
cha
haf
aft
ft 
t u
 un
und
nd 
d w
 wh
whr
hru
run
ung
ng 
g z
 zu
zum
um 
m s
 si
sie
ieb
ebt
bte
ten
en 
n b
 be
ber
eri
ric
ich
cht
ht 
t d
 de


In [ ]:
def constructTriGram(line):
    length = len(line)
    listOfGrams = []
    for i in range(length):
        c = []
        if i is 0 or i is 1:
            continue
        else:
            enterSequence(line[i-2:i+1])

In [286]:
ord('.')%97

46

In [282]:
def enterSequence(sequence):
    seq=[]
    print(sequence)
    seq.append(ord(sequence[0])%97)
    seq.append(ord(sequence[1])%97)
    seq.append(ord(sequence[2])%97)
    seq = getascii(seq)
#     print(seq)
    triGram[seq[0]][seq[1]][seq[2]] += 1

In [287]:
def getascii(seq):
    for i in range(len(seq)):
        if seq[i] is 32:
            seq[i] = 26
        if seq[i] == 46:
            seq[i] = 27
        if seq[i] is 48:
            seq[i] = 28
    return seq

In [289]:
def getBigram(freq_matrix):
    bi_grams = np.zeros((dimension,dimension))
    for i in range(dimension):
        for j in range(dimension):
            bi_grams[i][j] = sum(freq_matrix[i][j])
    return(bi_grams)

In [290]:
def getUnigram(bi_grams):
    uni_grams = np.zeros((dimension))
    for i in range(dimension):
        uni_grams[i] = sum(bi_grams[i])
    return(uni_grams)

In [291]:
D = 0.75
charCounts = dimension
bi_gram_count = getBigram(triGram)
uni_gram_count = getUnigram(bi_gram_count)
uni_gram = (uni_gram_count / sum(uni_gram_count))
bi_gram = np.zeros((dimension,dimension))
tri_gram = np.zeros((dimension,dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
#         if bi_gram_count[i][j] > 0:
        bi_gram[i][j] = ( (bi_gram_count[i][j] - D) / uni_gram_count[i] ) + (((D * charCounts) / uni_gram_count[i]) * uni_gram[j])
# print(bi_gram)
for w1 in range(dimension):
    for w2 in range(dimension):
        for w3 in range(dimension):
            if bi_gram_count[w1][w2] > 0:
                tri_gram[w1][w2][w3] = ((triGram[w1][w2][w3] - D) / bi_gram_count[w1][w2]) + ((D * charCounts) / bi_gram_count[w1][w2]) * bi_gram[w2][w3]
            else:
                tri_gram[w1][w2][w3] = ((triGram[w1][w2][w3] - D) ) + ((D * charCounts)) * bi_gram[w2][w3]
#             print(tri_gram[w1][w2][w3] - D)
print(tri_gram)


[[[ -8.25592442e-03   4.55277805e-03   1.64265288e-03 ...,   6.55046117e-03
    -1.14751962e-02  -1.11685170e-02]
  [  2.76968905e-04   1.34091724e-02   8.03455816e-04 ...,   3.11396512e-02
     1.01633877e-02  -2.36189834e-03]
  [ -2.77208593e-03  -2.84969964e-03  -2.82381305e-03 ...,  -2.70568570e-03
    -2.87863014e-03  -2.70412028e-03]
  ..., 
  [  3.02130910e-02   2.92591621e-02  -1.96375005e-03 ...,   1.22898776e-02
    -2.10010452e-03   3.93835034e-03]
  [ -3.28552049e-01  -4.70058783e-01  -4.22721531e-01 ...,   1.03829752e+01
    -3.14348010e-01   8.89264747e-01]
  [ -9.29713536e-02  -9.53435510e-02  -6.28444572e-02 ...,   4.78040037e-01
    -2.53589681e-02   2.86940823e+00]]

 [[ -5.70888390e-03   3.14819759e-03   1.13587699e-03 ...,   4.52957421e-03
    -7.93497611e-03  -7.72291067e-03]
  [  1.01772601e+00  -1.41812968e-01  -1.48578828e-01 ...,  -7.06232330e-03
    -1.49543186e-01  -1.51161494e-01]
  [ -7.26286513e-01  -7.46621306e-01  -7.39839018e-01 ...,   2.91110346e-01
  

In [ ]:
# maxi = 0
# i=0
# j=0
# k=0
# for w1 in range(38):
#     for w2 in range(38):
#         for w3 in range(38):
#             if abs(tri_gram[w1][w2][w3]) > maxi:
#                 maxi = abs(tri_gram[w1][w2][w3])
#                 i = w1
#                 j = w2
#                 k = w3
# print(maxi)
# from numpy import unravel_index
# unravel_index(triGram.argmax(), triGram.shape)
# triGram[26][19][8]              

In [ ]:
# unravel_index(tri_gram[26][19].argmax(), tri_gram[26][19].shape)
tri_gram[3][13]

In [ ]:
# class sequence(object):
#     def __init__(self, char="", probability=0):
#         self.char = char[0]
#         self.bigram = bigram(char[1])
#         self.trigram = trigram(char[2], probability)
        
#     def printTrigram(self):
#         print(self.char + self.bigram.printChar()+ self.trigram.printChar())

In [ ]:
# class bigram(self, char = '', probablility):
#     def __init__(self, char='', probability =0):
#         self.char = [char, probability]
#         self.gram =

In [ ]:
print((tri_gram[4][26].argmax()))

In [296]:
def generateText():
    temp = 0
    first = 0
    second = 0
    for i in range(500):
        print(asciiToChars(tri_gram[first][second].argmax()))
        temp = tri_gram[first][second].argmax()
        first = second
        second = temp
#         print('----1')
#         print(first)
#         print('----2')
#         print(second)
#         print('------------')
generateText()

t
e
n
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 
d
e
r
 


In [ ]:
def asciiToChars(value):
    char =''
    if value < 26:
        char = chr(int(value+97))
    elif value == 26:
        char = ' '
    elif value == 27:
        char = '.'
    elif value == 28:
        char = '0'
#     elif (value > 26 & value < 37):
#         char = chr(int(value+21))
    return char

In [ ]:
for i in range(28):
    for j in range(28):
        print(tri_gram[i][j].argmax())

In [ ]:
asciiToChars(27)

In [ ]:
triGram = np.zeros((38,38,38))
constructTriGram("resumption of the session")